In [ ]:
import pandas as pd
import numpy as np

# 1. 데이터 로드
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

def calculate_prospect_score(df):
    # (1) 공격 기여도 (Attacking Contribution)
    # 결정력과 슛 파워, 그리고 스트라이커로서의 기본 레이팅을 종합
    df['Attacking_Score'] = df[['Finishing', 'ShotPower', 'STRating', 'Positioning']].mean(axis=1)
    
    # (2) 기동성 및 기술 (Technique & Pace)
    # 현대 공격수에게 필수적인 속도와 드리블 능력
    df['Technique_Score'] = df[['PaceTotal', 'DribblingTotal', 'BallControl', 'Agility']].mean(axis=1)
    
    # (3) 하드웨어 및 멘탈 (Hardware & Mental)
    # 몸싸움 능력과 압박 속에서의 침착함
    df['Physical_Mental'] = df[['Strength', 'Reactions', 'Composure', 'Stamina']].mean(axis=1)
    
    # (4) 나이 가산점 (Youth Multiplier) - 가장 중요!
    # 같은 스탯이라도 나이가 어릴수록 잠재력 점수를 폭발적으로 부여
    # 17세와 23세의 차이를 크게 벌리기 위해 로그 또는 제곱근을 활용할 수 있음
    df['Age_Efficiency'] = (df['Attacking_Score'] + df['Technique_Score'] + df['Physical_Mental']) / (df['Age'] - 14) # 15세 기준 기점
    
    # [최종 점수] 모든 점수를 합산하고 나이 효율성을 곱함
    # 가중치를 주어 가장 중요한 지표를 강조
    df['Final_Prospect_Score'] = (
        df['Attacking_Score'] * 0.4 + 
        df['Technique_Score'] * 0.3 + 
        df['Physical_Mental'] * 0.3
    ) * (1 / np.log1p(df['Age'] - 15)) # 나이가 많아질수록 점수가 깎이는 구조
    
    return df

# 지표 계산 적용
test_scored = calculate_prospect_score(test)

# 2. 공격수 포지션 필터링
# ST(스트라이커), LW/RW(윙어), CF(센터포워드)
fw_positions = ['ST', 'LW', 'RW', 'CF', 'LF', 'RF', 'LS', 'RS']
attackers = test_scored[test_scored['Position'].isin(fw_positions)].copy()

# 3. 유망주 TOP 10 추출 (Final_Prospect_Score 기준)
# 너무 나이가 많은 선수는 제외 (예: 23세 이하만)
young_attackers = attackers[attackers['Age'] <= 23]
top_10_prospects = young_attackers.sort_values(by='Final_Prospect_Score', ascending=False).head(10)

# 4. 결과 출력
print("\n" + "="*90)
print("📊 [데이터 지표 기반] AI 모델 없이 선정한 차세대 공격수 TOP 10")
print("-" * 90)
output_cols = ['ID', 'Age', 'Position', 'Finishing', 'STRating', 'PaceTotal', 'Reactions', 'Final_Prospect_Score']
print(top_10_prospects[output_cols].to_string(index=False))
print("="*90)


🏆차세대 공격수 유망주 TOP 10
-------------------------------------------------------------------------------------
       ID  Age Position  Finishing  STRating  Offensive_IQ  Physical_Potential  Prospect_Prob
TEST_1548   17       ST     5.9341    5.2055      4.566433            5.946325       0.992952
TEST_1224   16       ST     5.7143    5.2055      3.543667            5.240875       0.992912
TEST_1066   16       ST     7.1429    5.8904      5.181267            3.765575       0.989161
TEST_0191   18       ST     6.0440    5.4795      4.601900            5.463100       0.985901
TEST_0930   17       ST     5.6044    5.0685      4.103400            6.316525       0.984000
TEST_0342   18       ST     6.1538    5.2055      4.513900            5.581350       0.983743
TEST_0139   17       ST     6.0440    4.9315      4.251433            5.143750       0.983161
TEST_1327   18       ST     5.7143    5.2055      4.243267            5.227075       0.982973
TEST_0322   17       ST     6.1538    5.3425   